# Convert STIX Examples to Data Form Templates

## Overview

This notebook systematically analyzes all STIX example files in the `Block_Families/examples` directory and converts them into data form templates suitable for the Brett Blocks template-driven architecture.

### Key Objectives:
1. **Parse Example Files**: Load and analyze all STIX examples
2. **Identify Embedded References**: Find `_ref` and `_refs` fields that need to be extracted
3. **Create Data Forms**: Generate clean templates without embedded references
4. **Map to Directories**: Determine correct placement in StixORM structure
5. **Generate Sequences**: Document object creation order for complex dependencies

### Template-Driven Architecture Principles:
- **Data Forms**: Contain only direct object properties (no embedded references)
- **Python Blocks**: Receive data form + separate reference objects as parameters
- **Foreign Keys**: Handled as function parameters, not embedded in data forms

## 1. Import Required Libraries

In [1]:
import json
import os
from pathlib import Path
import re
from collections import defaultdict
from typing import Dict, List, Any, Tuple
import copy

# Set up paths - we're in Orchestration directory, need to go up one level
base_path = Path.cwd().parent  # Go up from Orchestration to project root
examples_path = base_path / "Block_Families" / "examples"
stixorm_path = base_path / "Block_Families" / "StixORM"
orchestration_path = base_path / "Orchestration"

print(f"Base path: {base_path}")
print(f"Examples path: {examples_path}")
print(f"StixORM path: {stixorm_path}")
print(f"Orchestration path: {orchestration_path}")

# Verify paths exist
print(f"\nPath verification:")
print(f"Examples directory exists: {examples_path.exists()}")
print(f"StixORM directory exists: {stixorm_path.exists()}")
print(f"Orchestration directory exists: {orchestration_path.exists()}")

Base path: c:\projects\brett_blocks
Examples path: c:\projects\brett_blocks\Block_Families\examples
StixORM path: c:\projects\brett_blocks\Block_Families\StixORM
Orchestration path: c:\projects\brett_blocks\Orchestration

Path verification:
Examples directory exists: True
StixORM directory exists: True
Orchestration directory exists: True


## 2. Load and Parse Example Files

In [2]:
def load_example_files():
    """Load all JSON files from the examples directory"""
    example_files = {}
    stix_objects_by_type = defaultdict(list)
    
    # Get all JSON files
    json_files = list(examples_path.glob("*.json"))
    print(f"Found {len(json_files)} JSON files in examples directory")
    
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = json.load(f)
            
            example_files[file_path.name] = content
            
            # Organize by STIX type
            if isinstance(content, list):
                for obj in content:
                    if isinstance(obj, dict) and 'type' in obj:
                        stix_objects_by_type[obj['type']].append({
                            'file': file_path.name,
                            'object': obj
                        })
            elif isinstance(content, dict) and 'type' in content:
                stix_objects_by_type[content['type']].append({
                    'file': file_path.name,
                    'object': content
                })
                
        except Exception as e:
            print(f"Error loading {file_path.name}: {e}")
    
    return example_files, stix_objects_by_type

# Load all example files
example_files, stix_objects_by_type = load_example_files()

print(f"\nLoaded {len(example_files)} example files")
print(f"Found {len(stix_objects_by_type)} unique STIX types:")
for stix_type, objects in stix_objects_by_type.items():
    print(f"  {stix_type}: {len(objects)} objects")

Found 60 JSON files in examples directory

Loaded 60 example files
Found 37 unique STIX types:
  attack-pattern: 1 objects
  intrusion-set: 2 objects
  relationship: 7 objects
  identity: 2 objects
  indicator: 1 objects
  malware: 4 objects
  artifact: 3 objects
  autonomous-system: 1 objects
  campaign: 1 objects
  course-of-action: 1 objects
  directory: 3 objects
  ipv4-addr: 15 objects
  domain-name: 3 objects
  email-addr: 8 objects
  email-message: 3 objects
  file: 15 objects
  grouping: 1 objects
  incident: 1 objects
  infrastructure: 1 objects
  ipv6-addr: 3 objects
  location: 3 objects
  mac-addr: 1 objects
  malware-analysis: 1 objects
  mutex: 1 objects
  network-traffic: 7 objects
  note: 1 objects
  observed-data: 1 objects
  opinion: 1 objects
  process: 3 objects
  software: 1 objects
  threat-actor: 1 objects
  tool: 1 objects
  url: 1 objects
  user-account: 3 objects
  vulnerability: 1 objects
  windows-registry-key: 2 objects
  x509-certificate: 2 objects


## 3. Analyze STIX Object Structure

In [3]:
def analyze_object_structure(obj):
    """Analyze a STIX object to understand its structure and properties"""
    analysis = {
        'type': obj.get('type'),
        'properties': list(obj.keys()),
        'embedded_refs': [],
        'standard_props': [],
        'custom_props': []
    }
    
    # Standard STIX properties that appear in most objects
    standard_stix_props = {
        'type', 'spec_version', 'id', 'created', 'modified', 
        'created_by_ref', 'revoked', 'labels', 'confidence',
        'lang', 'external_references', 'object_marking_refs',
        'granular_markings'
    }
    
    for prop, value in obj.items():
        if prop.endswith('_ref') or prop.endswith('_refs'):
            analysis['embedded_refs'].append({
                'property': prop,
                'value': value,
                'is_list': isinstance(value, list)
            })
        elif prop in standard_stix_props:
            analysis['standard_props'].append(prop)
        else:
            analysis['custom_props'].append(prop)
    
    return analysis

# Analyze all objects
object_analyses = {}
for stix_type, objects in stix_objects_by_type.items():
    object_analyses[stix_type] = []
    for obj_info in objects:
        analysis = analyze_object_structure(obj_info['object'])
        analysis['source_file'] = obj_info['file']
        object_analyses[stix_type].append(analysis)

# Display analysis summary
print("STIX Object Structure Analysis:")
print("=" * 50)
for stix_type, analyses in object_analyses.items():
    print(f"\n{stix_type.upper()} ({len(analyses)} objects):")
    
    # Collect all properties across objects of this type
    all_props = set()
    all_refs = set()
    for analysis in analyses:
        all_props.update(analysis['properties'])
        all_refs.update([ref['property'] for ref in analysis['embedded_refs']])
    
    print(f"  Properties: {sorted(all_props)}")
    if all_refs:
        print(f"  Embedded Refs: {sorted(all_refs)}")
    else:
        print(f"  Embedded Refs: None")

STIX Object Structure Analysis:

ATTACK-PATTERN (1 objects):
  Properties: ['created', 'description', 'external_references', 'id', 'modified', 'name', 'spec_version', 'type']
  Embedded Refs: None

INTRUSION-SET (2 objects):
  Properties: ['aliases', 'created', 'created_by_ref', 'description', 'goals', 'id', 'modified', 'name', 'spec_version', 'type']
  Embedded Refs: ['created_by_ref']

RELATIONSHIP (7 objects):
  Properties: ['created', 'created_by_ref', 'id', 'modified', 'relationship_type', 'source_ref', 'spec_version', 'target_ref', 'type']
  Embedded Refs: ['created_by_ref', 'source_ref', 'target_ref']

IDENTITY (2 objects):
  Properties: ['created', 'created_by_ref', 'description', 'id', 'identity_class', 'modified', 'name', 'spec_version', 'type']
  Embedded Refs: ['created_by_ref']

INDICATOR (1 objects):
  Properties: ['created', 'created_by_ref', 'description', 'id', 'indicator_types', 'modified', 'name', 'pattern', 'pattern_type', 'spec_version', 'type', 'valid_from']
  Emb

## 4. Identify Embedded References

In [4]:
def identify_reference_patterns():
    """Identify all embedded reference patterns across all STIX objects"""
    ref_patterns = defaultdict(list)
    object_dependencies = defaultdict(list)
    
    for stix_type, analyses in object_analyses.items():
        for analysis in analyses:
            for ref in analysis['embedded_refs']:
                ref_patterns[ref['property']].append({
                    'stix_type': stix_type,
                    'file': analysis['source_file'],
                    'is_list': ref['is_list'],
                    'value': ref['value']
                })
                
                # Track dependencies
                if ref['is_list'] and isinstance(ref['value'], list):
                    for ref_id in ref['value']:
                        if isinstance(ref_id, str) and '--' in ref_id:
                            target_type = ref_id.split('--')[0]
                            object_dependencies[stix_type].append({
                                'depends_on': target_type,
                                'property': ref['property'],
                                'ref_id': ref_id
                            })
                elif isinstance(ref['value'], str) and '--' in ref['value']:
                    target_type = ref['value'].split('--')[0]
                    object_dependencies[stix_type].append({
                        'depends_on': target_type,
                        'property': ref['property'],
                        'ref_id': ref['value']
                    })
    
    return ref_patterns, object_dependencies

ref_patterns, object_dependencies = identify_reference_patterns()

print("EMBEDDED REFERENCE PATTERNS:")
print("=" * 50)
for ref_prop, usages in ref_patterns.items():
    print(f"\n{ref_prop}:")
    stix_types = set([usage['stix_type'] for usage in usages])
    print(f"  Used by: {sorted(stix_types)}")
    print(f"  Total usages: {len(usages)}")
    
    # Show example values
    example_values = set()
    for usage in usages[:3]:  # Show first 3 examples
        if isinstance(usage['value'], list):
            example_values.add(f"[{len(usage['value'])} items]")
        else:
            example_values.add(str(usage['value'])[:50])
    print(f"  Example values: {list(example_values)}")

print("\n\nOBJECT DEPENDENCIES:")
print("=" * 50)
for obj_type, deps in object_dependencies.items():
    depends_on = set([dep['depends_on'] for dep in deps])
    print(f"{obj_type} depends on: {sorted(depends_on)}")

EMBEDDED REFERENCE PATTERNS:

created_by_ref:
  Used by: ['campaign', 'course-of-action', 'grouping', 'identity', 'incident', 'indicator', 'intrusion-set', 'location', 'malware', 'observed-data', 'opinion', 'relationship', 'threat-actor', 'tool', 'vulnerability']
  Total usages: 18
  Example values: ['identity--e5f1b90a-d9b6-40ab-81a9-8a29df4b6b65']

source_ref:
  Used by: ['relationship']
  Total usages: 7
  Example values: ['intrusion-set--0c7e22ad-b099-4dc3-b0df-2ea3f49ae2e', 'indicator--8e2e2d2b-17d4-4cbf-938f-98ee46b3cd3f', 'course-of-action--8e2e2d2b-17d4-4cbf-938f-98ee46b3']

target_ref:
  Used by: ['relationship']
  Total usages: 7
  Example values: ['malware--0c7b5b88-8ff7-4a4d-aa9d-feb398cd0061', 'malware--31b940d4-6f7f-459a-80ea-9c1f17b5891b', 'attack-pattern--7e33a43e-e34b-40ec-89da-36c9bb2cac']

sample_refs:
  Used by: ['malware']
  Total usages: 1
  Example values: ['[1 items]']

resolves_to_refs:
  Used by: ['domain-name']
  Total usages: 2
  Example values: ['[1 items]'

## 5. Extract Data Form Templates

In [5]:
def create_data_form_template(stix_obj):
    """Create a data form template by removing embedded references"""
    data_form = copy.deepcopy(stix_obj)
    extracted_refs = {}
    
    # Remove standard STIX metadata that should be auto-generated
    auto_generated_props = ['id', 'spec_version', 'created', 'modified']
    for prop in auto_generated_props:
        if prop in data_form:
            del data_form[prop]
    
    # Extract embedded references
    props_to_remove = []
    for prop, value in data_form.items():
        if prop.endswith('_ref') or prop.endswith('_refs'):
            extracted_refs[prop] = value
            props_to_remove.append(prop)
    
    # Remove embedded references from data form
    for prop in props_to_remove:
        del data_form[prop]
    
    return data_form, extracted_refs

def get_stix_type_mapping():
    """Map STIX types to their expected directory structure"""
    
    # Load the inventory from the documentation
    inventory = {
        # SDO types (currently implemented)
        'identity': 'SDO/Identity',
        'indicator': 'SDO/Indicator', 
        'impact': 'SDO/Impact',
        'incident': 'SDO/Incident',
        'event': 'SDO/Event',
        'observed-data': 'SDO/Observed_Data',
        'sequence': 'SDO/Sequence',
        'task': 'SDO/Task',
        
        # SCO types (currently implemented)
        'anecdote': 'SCO/Anecdote',
        'email-addr': 'SCO/Email_Addr',
        'user-account': 'SCO/User_Account',
        'url': 'SCO/URL',
        'email-message': 'SCO/Email_Message',
        
        # SRO types (currently implemented)
        'relationship': 'SRO/Relationship',
        'sighting': 'SRO/Sighting',
        
        # Standard STIX 2.1 SDO types (templates exist)
        'attack-pattern': 'SDO/Attack_Pattern',
        'campaign': 'SDO/Campaign',
        'course-of-action': 'SDO/Course_of_Action',
        'grouping': 'SDO/Grouping',
        'infrastructure': 'SDO/Infrastructure',
        'intrusion-set': 'SDO/Instrusion_Set',
        'location': 'SDO/Location',
        'malware-analysis': 'SDO/Malware_Analysis',
        'note': 'SDO/Note',
        'opinion': 'SDO/Opinion',
        'report': 'SDO/Report',
        'threat-actor': 'SDO/Threat_Actor',
        'vulnerability': 'SDO/Vulnerability',
        
        # Standard STIX 2.1 SCO types (templates exist)
        'artifact': 'SCO/Artifact',
        'autonomous-system': 'SCO/Autonomous_System',
        'directory': 'SCO/Directory',
        'domain-name': 'SCO/Domain_Name',
        'file': 'SCO/File',
        'ipv4-addr': 'SCO/IPv4_Addr',
        'ipv6-addr': 'SCO/IPv6_Addr',
        'mac-addr': 'SCO/MAC_Address',
        'mutex': 'SCO/Mutex',
        'software': 'SCO/Software',
        'x509-certificate': 'SCO/X509_Cert'
    }
    
    return inventory

# Create data form templates for all objects
stix_type_mapping = get_stix_type_mapping()
data_form_templates = {}
extracted_references = {}
mapping_results = {}

for stix_type, objects in stix_objects_by_type.items():
    print(f"\nProcessing {stix_type} ({len(objects)} objects):")
    
    data_form_templates[stix_type] = []
    extracted_references[stix_type] = []
    
    # Check if we have a directory mapping for this type
    directory_path = stix_type_mapping.get(stix_type)
    mapping_results[stix_type] = {
        'has_directory': directory_path is not None,
        'directory_path': directory_path,
        'object_count': len(objects)
    }
    
    for i, obj_info in enumerate(objects):
        obj = obj_info['object']
        source_file = obj_info['file']
        
        # Create data form template
        data_form, refs = create_data_form_template(obj)
        
        template_info = {
            'source_file': source_file,
            'object_index': i,
            'data_form': data_form,
            'extracted_refs': refs,
            'directory_path': directory_path
        }
        
        data_form_templates[stix_type].append(template_info)
        
        print(f"  {source_file}[{i}]: {len(refs)} refs extracted")

print(f"\nDirectory Mapping Results:")
print("=" * 50)
for stix_type, result in mapping_results.items():
    status = "✅ Mapped" if result['has_directory'] else "❌ No Directory"
    print(f"{stix_type}: {status} ({result['object_count']} objects)")
    if result['has_directory']:
        print(f"   → {result['directory_path']}")


Processing attack-pattern (1 objects):
  aaa_attack_pattern.json[0]: 0 refs extracted

Processing intrusion-set (2 objects):
  aaa_attack_pattern.json[0]: 0 refs extracted
  intrusion_set.json[1]: 1 refs extracted

Processing relationship (7 objects):
  aaa_attack_pattern.json[0]: 2 refs extracted
  aaa_indicator.json[1]: 3 refs extracted
  course_action.json[2]: 3 refs extracted
  infrastructure.json[3]: 2 refs extracted
  infrastructure.json[4]: 2 refs extracted
  infrastructure.json[5]: 2 refs extracted
  malware_analysis.json[6]: 2 refs extracted

Processing identity (2 objects):
  aaa_identity.json[0]: 0 refs extracted
  aaa_identity.json[1]: 1 refs extracted

Processing indicator (1 objects):
  aaa_indicator.json[0]: 1 refs extracted

Processing malware (4 objects):
  aaa_indicator.json[0]: 1 refs extracted
  aaa_malware.json[1]: 0 refs extracted
  infrastructure.json[2]: 0 refs extracted
  malware_analysis.json[3]: 1 refs extracted

Processing artifact (3 objects):
  artifact_b

## 6. Generate Object Creation Sequences

In [6]:
def generate_creation_sequences():
    """Generate object creation sequences based on dependencies"""
    sequences = {}
    
    # Analyze each example file to determine creation sequences
    for filename, content in example_files.items():
        if isinstance(content, list) and len(content) > 1:
            sequence = {
                'file': filename,
                'objects': [],
                'dependencies': [],
                'creation_order': []
            }
            
            # Map object IDs to their types and positions
            id_to_info = {}
            for i, obj in enumerate(content):
                if isinstance(obj, dict) and 'id' in obj and 'type' in obj:
                    id_to_info[obj['id']] = {
                        'type': obj['type'],
                        'index': i,
                        'object': obj
                    }
            
            # Analyze dependencies within this file
            for i, obj in enumerate(content):
                if isinstance(obj, dict) and 'type' in obj:
                    obj_info = {
                        'index': i,
                        'type': obj['type'],
                        'id': obj.get('id'),
                        'depends_on': []
                    }
                    
                    # Find references to other objects in the same file
                    for prop, value in obj.items():
                        if prop.endswith('_ref') or prop.endswith('_refs'):
                            if isinstance(value, list):
                                for ref_id in value:
                                    if ref_id in id_to_info:
                                        obj_info['depends_on'].append({
                                            'ref_id': ref_id,
                                            'ref_type': id_to_info[ref_id]['type'],
                                            'property': prop
                                        })
                            elif isinstance(value, str) and value in id_to_info:
                                obj_info['depends_on'].append({
                                    'ref_id': value,
                                    'ref_type': id_to_info[value]['type'],
                                    'property': prop
                                })
                    
                    sequence['objects'].append(obj_info)
            
            # Determine creation order using topological sort
            def topological_sort(objects):
                # Create dependency graph
                deps = {obj['index']: set() for obj in objects}
                for obj in objects:
                    for dep in obj['depends_on']:
                        dep_index = id_to_info[dep['ref_id']]['index']
                        deps[obj['index']].add(dep_index)
                
                # Kahn's algorithm
                in_degree = {i: 0 for i in deps.keys()}
                for dependents in deps.values():
                    for dep in dependents:
                        in_degree[dep] += 1
                
                queue = [i for i in in_degree.keys() if in_degree[i] == 0]
                result = []
                
                while queue:
                    current = queue.pop(0)
                    result.append(current)
                    
                    for dependent in deps[current]:
                        in_degree[dependent] -= 1
                        if in_degree[dependent] == 0:
                            queue.append(dependent)
                
                return result
            
            if len(sequence['objects']) > 1:
                creation_order = topological_sort(sequence['objects'])
                sequence['creation_order'] = creation_order
                sequences[filename] = sequence
    
    return sequences

# Generate sequences
creation_sequences = generate_creation_sequences()

print("OBJECT CREATION SEQUENCES:")
print("=" * 50)
for filename, sequence in creation_sequences.items():
    print(f"\n{filename}:")
    print(f"  Objects: {len(sequence['objects'])}")
    
    for order_idx, obj_idx in enumerate(sequence['creation_order']):
        obj = sequence['objects'][obj_idx]
        deps_text = ""
        if obj['depends_on']:
            dep_types = [dep['ref_type'] for dep in obj['depends_on']]
            deps_text = f" (depends on: {', '.join(dep_types)})"
        print(f"    {order_idx + 1}. {obj['type']}{deps_text}")

# Analyze common patterns
print(f"\n\nSEQUENCE ANALYSIS:")
print("=" * 50)
print(f"Files with multiple objects: {len(creation_sequences)}")

dependency_patterns = defaultdict(int)
for sequence in creation_sequences.values():
    for obj in sequence['objects']:
        for dep in obj['depends_on']:
            pattern = f"{obj['type']} → {dep['ref_type']}"
            dependency_patterns[pattern] += 1

print(f"\nCommon dependency patterns:")
for pattern, count in sorted(dependency_patterns.items(), key=lambda x: x[1], reverse=True):
    print(f"  {pattern}: {count} times")

OBJECT CREATION SEQUENCES:

aaa_attack_pattern.json:
  Objects: 3
    1. relationship (depends on: intrusion-set, attack-pattern)
    2. attack-pattern
    3. intrusion-set

aaa_identity.json:
  Objects: 2
    1. identity (depends on: identity)
    2. identity

aaa_indicator.json:
  Objects: 3
    1. relationship (depends on: indicator, malware)
    2. indicator
    3. malware

course_action.json:
  Objects: 2
    1. relationship (depends on: course-of-action)
    2. course-of-action

domain.json:
  Objects: 2
    1. domain-name (depends on: ipv4-addr)
    2. ipv4-addr

email_headers.json:
  Objects: 3
    1. email-message (depends on: email-addr, email-addr)
    2. email-addr
    3. email-addr

email_mime.json:
  Objects: 6
    1. artifact
    2. file
    3. email-message (depends on: email-addr, email-addr, email-addr)
    4. email-addr
    5. email-addr
    6. email-addr

email_simple.json:
  Objects: 3
    1. email-message (depends on: email-addr, email-addr)
    2. email-addr
    

## 7. Create Directory Structure Mapping

In [9]:
def save_data_form_templates():
    """Save data form templates to their appropriate directories"""
    saved_files = []
    
    for stix_type, templates in data_form_templates.items():
        # Skip types without directory mapping
        directory_path = stix_type_mapping.get(stix_type)
        if not directory_path:
            print(f"Skipping {stix_type} - no directory mapping")
            continue
        
        target_dir = stixorm_path / directory_path
        
        # Create directory if it doesn't exist
        target_dir.mkdir(parents=True, exist_ok=True)
        
        for i, template_info in enumerate(templates):
            # Generate filename based on source and index
            source_base = template_info['source_file'].replace('.json', '')
            if len(templates) > 1:
                filename = f"{source_base}_{i+1}_dataform.json"
            else:
                filename = f"{source_base}_dataform.json"
            
            file_path = target_dir / filename
            
            # Save the data form template
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(template_info['data_form'], f, indent=2)
            
            saved_files.append({
                'stix_type': stix_type,
                'file_path': str(file_path),
                'source_file': template_info['source_file'],
                'refs_extracted': len(template_info['extracted_refs'])
            })
            
            print(f"Saved: {file_path}")
    
    return saved_files

# Actually save the data form template files
print("SAVING DATA FORM TEMPLATE FILES:")
print("=" * 50)

saved_files = save_data_form_templates()

print(f"\nSAVED FILES SUMMARY:")
print(f"  Total files saved: {len(saved_files)}")

# Group by STIX type for summary
saved_by_type = {}
for file_info in saved_files:
    stix_type = file_info['stix_type']
    if stix_type not in saved_by_type:
        saved_by_type[stix_type] = []
    saved_by_type[stix_type].append(file_info)

for stix_type, files in saved_by_type.items():
    print(f"  {stix_type}: {len(files)} files")

# Also show what would be saved for unmappable types
print(f"\nUNMAPPABLE TYPES (not saved):")
total_mappable = 0
total_unmappable = 0

for stix_type, templates in data_form_templates.items():
    directory_path = stix_type_mapping.get(stix_type)
    
    if directory_path:
        total_mappable += len(templates)
    else:
        print(f"  ❌ {stix_type} - No directory mapping ({len(templates)} templates)")
        total_unmappable += len(templates)

print(f"\nFINAL SUMMARY:")
print(f"  Mappable templates saved: {total_mappable}")
print(f"  Unmappable templates: {total_unmappable}")
print(f"  Total templates: {total_mappable + total_unmappable}")

# Show unmappable types that need new directories
unmappable_types = [stix_type for stix_type in data_form_templates.keys() 
                   if stix_type not in stix_type_mapping]
if unmappable_types:
    print(f"\nTYPES NEEDING NEW DIRECTORIES:")
    for stix_type in sorted(unmappable_types):
        count = len(data_form_templates[stix_type])
        print(f"  {stix_type} ({count} templates) - needs StixORM directory")

SAVING DATA FORM TEMPLATE FILES:
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SDO\Attack_Pattern\aaa_attack_pattern_dataform.json
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SDO\Instrusion_Set\aaa_attack_pattern_1_dataform.json
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SDO\Instrusion_Set\intrusion_set_2_dataform.json
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SRO\Relationship\aaa_attack_pattern_1_dataform.json
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SRO\Relationship\aaa_indicator_2_dataform.json
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SRO\Relationship\course_action_3_dataform.json
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SRO\Relationship\infrastructure_4_dataform.json
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SRO\Relationship\infrastructure_5_dataform.json
Saved: c:\projects\brett_blocks\Block_Families\StixORM\SRO\Relationship\infrastructure_6_dataform.json
Saved: c:\projects\brett_bl

## 8. Generate Orchestration Documentation

## 7.5. Generate Reconstitution Instructions

Generate comprehensive instructions for reconstructing original embedded reference structures using the Python blocks and data form templates.

In [13]:
def generate_reconstitution_instructions():
    """
    Generate comprehensive instructions for reconstructing original STIX objects
    using Python blocks and data form templates.
    """
    reconstitution_specs = {}
    
    # Process each example file
    for filename, content in example_files.items():
        if isinstance(content, list):
            # Multi-object files require sequencing
            file_spec = {
                'file': filename,
                'type': 'multi_object',
                'objects': [],
                'python_block_calls': []
            }
            
            # Map objects and their data forms
            for i, obj in enumerate(content):
                if isinstance(obj, dict) and 'type' in obj:
                    obj_id = obj.get('id', 'object_' + str(i))
                    obj_type = obj['type']
                    
                    # Find corresponding data form template
                    data_form_filename = None
                    extracted_refs = {}
                    
                    if obj_type in data_form_templates:
                        for template_info in data_form_templates[obj_type]:
                            if template_info['source_file'] == filename:
                                # Generate the data form filename
                                source_base = template_info['source_file'].replace('.json', '')
                                template_count = len([t for t in data_form_templates[obj_type] 
                                                    if t['source_file'] == filename])
                                if template_count > 1:
                                    # Find which index this is
                                    matching_templates = [j for j, t in enumerate(data_form_templates[obj_type]) 
                                                        if t['source_file'] == filename and 
                                                        t['object_index'] == i]
                                    if matching_templates:
                                        template_index = matching_templates[0] + 1
                                        data_form_filename = source_base + '_' + str(template_index) + '_dataform.json'
                                    else:
                                        # Fallback if no exact match found
                                        data_form_filename = source_base + '_' + str(i+1) + '_dataform.json'
                                else:
                                    data_form_filename = source_base + '_dataform.json'
                                
                                extracted_refs = template_info['extracted_refs']
                                break
                    
                    obj_info = {
                        'index': i,
                        'id': obj_id,
                        'type': obj_type,
                        'data_form_file': data_form_filename,
                        'directory_path': stix_type_mapping.get(obj_type),
                        'extracted_refs': extracted_refs
                    }
                    
                    file_spec['objects'].append(obj_info)
            
            reconstitution_specs[filename] = file_spec
            
        else:
            # Single object files
            if isinstance(content, dict) and 'type' in content:
                obj_type = content['type']
                obj_id = content.get('id', 'single_object')
                
                # Find data form template
                data_form_filename = None
                extracted_refs = {}
                
                if obj_type in data_form_templates:
                    for template_info in data_form_templates[obj_type]:
                        if template_info['source_file'] == filename:
                            source_base = template_info['source_file'].replace('.json', '')
                            data_form_filename = source_base + '_dataform.json'
                            extracted_refs = template_info['extracted_refs']
                            break
                
                file_spec = {
                    'file': filename,
                    'type': 'single_object',
                    'object_id': obj_id,
                    'object_type': obj_type,
                    'data_form_file': data_form_filename,
                    'directory_path': stix_type_mapping.get(obj_type),
                    'extracted_refs': extracted_refs
                }
                
                reconstitution_specs[filename] = file_spec
    
    return reconstitution_specs

# Generate reconstitution specifications
print("GENERATING RECONSTITUTION INSTRUCTIONS:")
print("=" * 50)

reconstitution_specs = generate_reconstitution_instructions()

print("Generated reconstitution specs for {} files".format(len(reconstitution_specs)))

# Display summary
multi_object_files = [f for f, spec in reconstitution_specs.items() if spec['type'] == 'multi_object']
single_object_files = [f for f, spec in reconstitution_specs.items() if spec['type'] == 'single_object']

print("")
print("File types:")
print("  Multi-object files: {}".format(len(multi_object_files)))
print("  Single-object files: {}".format(len(single_object_files)))

print("")
print("Multi-object file details:")
for filename in multi_object_files:
    spec = reconstitution_specs[filename]
    objects = len(spec['objects'])
    print("  {}: {} objects".format(filename, objects))

print("")
print("Single objects with references:")
for filename in single_object_files:
    spec = reconstitution_specs[filename]
    if spec['extracted_refs']:
        refs = len(spec['extracted_refs'])
        print("  {}: {} references".format(filename, refs))

# Store specs for documentation generation
globals()['reconstitution_specs'] = reconstitution_specs

GENERATING RECONSTITUTION INSTRUCTIONS:
Generated reconstitution specs for 60 files

File types:
  Multi-object files: 60
  Single-object files: 0

Multi-object file details:
  aaa_attack_pattern.json: 3 objects
  aaa_identity.json: 2 objects
  aaa_indicator.json: 3 objects
  aaa_malware.json: 1 objects
  artifact_basic.json: 1 objects
  artifact_encrypted.json: 1 objects
  autonomous.json: 1 objects
  campaign.json: 1 objects
  course_action.json: 2 objects
  directory.json: 1 objects
  domain.json: 2 objects
  email_basic_addr.json: 1 objects
  email_headers.json: 3 objects
  email_mime.json: 6 objects
  email_simple.json: 3 objects
  file_archive_unencrypted.json: 4 objects
  file_basic.json: 1 objects
  file_basic_encoding.json: 1 objects
  file_basic_parent.json: 2 objects
  file_binary.json: 1 objects
  file_image_simple.json: 1 objects
  file_ntfs_stream.json: 1 objects
  file_pdf_basic.json: 1 objects
  grouping.json: 1 objects
  incident.json: 1 objects
  infrastructure.json: 

In [16]:
def generate_orchestration_documentation():
    """Generate comprehensive markdown documentation for orchestration"""
    from datetime import datetime
    
    doc_content = []
    doc_content.append("# STIX Examples to Data Forms Conversion Guide")
    doc_content.append("\n*Generated automatically from examples analysis*")
    doc_content.append(f"\n**Analysis Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    doc_content.append(f"**Total Example Files**: {len(example_files)}")
    doc_content.append(f"**STIX Object Types**: {len(stix_objects_by_type)}")
    
    # Overview section
    doc_content.append("\n## Overview")
    doc_content.append("\nThis document records the conversion of STIX examples from the `Block_Families/examples` directory into data form templates compatible with the Brett Blocks template-driven architecture.")
    
    doc_content.append("\n### Key Principles:")
    doc_content.append("- **Data Forms**: Clean templates with no embedded references")
    doc_content.append("- **Extracted References**: Handled as separate function parameters")
    doc_content.append("- **Object Sequences**: Dependency-ordered creation for complex scenarios")
    
    # Data form templates section
    doc_content.append("\n## Data Form Templates Generated")
    
    mappable_count = sum(len(templates) for stix_type, templates in data_form_templates.items() 
                        if stix_type in stix_type_mapping)
    unmappable_count = sum(len(templates) for stix_type, templates in data_form_templates.items() 
                          if stix_type not in stix_type_mapping)
    
    doc_content.append(f"\n**Summary**: {mappable_count} mappable templates, {unmappable_count} unmappable templates")
    
    # Mappable templates
    doc_content.append("\n### Successfully Mapped Templates")
    
    for stix_type in sorted(stix_type_mapping.keys()):
        if stix_type in data_form_templates:
            templates = data_form_templates[stix_type]
            directory_path = stix_type_mapping[stix_type]
            
            doc_content.append(f"\n#### {stix_type.upper()}")
            doc_content.append(f"**Directory**: `Block_Families/StixORM/{directory_path}`")
            doc_content.append(f"**Templates**: {len(templates)}")
            
            for i, template_info in enumerate(templates):
                source_base = template_info['source_file'].replace('.json', '')
                if len(templates) > 1:
                    filename = f"{source_base}_{i+1}_dataform.json"
                else:
                    filename = f"{source_base}_dataform.json"
                
                refs_count = len(template_info['extracted_refs'])
                doc_content.append(f"- `{filename}` (from {template_info['source_file']}, {refs_count} refs)")
    
    # Unmappable templates
    unmappable_types = [stix_type for stix_type in data_form_templates.keys() 
                       if stix_type not in stix_type_mapping]
    
    if unmappable_types:
        doc_content.append("\n### Templates Requiring New Directories")
        doc_content.append("\nThese STIX types need directory structures created:")
        
        for stix_type in sorted(unmappable_types):
            templates = data_form_templates[stix_type]
            doc_content.append(f"\n#### {stix_type.upper()}")
            doc_content.append(f"**Needs Directory**: `Block_Families/StixORM/???/{stix_type.replace('-', '_').title()}`")
            doc_content.append(f"**Templates**: {len(templates)}")
            
            for template_info in templates:
                refs_count = len(template_info['extracted_refs'])
                doc_content.append(f"- From `{template_info['source_file']}` ({refs_count} refs)")
    
    # Object creation sequences
    doc_content.append("\n## Object Creation Sequences")
    doc_content.append("\nFor examples with multiple interdependent objects:")
    
    for filename, sequence in creation_sequences.items():
        doc_content.append(f"\n### {filename}")
        doc_content.append(f"**Objects**: {len(sequence['objects'])}")
        doc_content.append(f"**Creation Order**:")
        
        for order_idx, obj_idx in enumerate(sequence['creation_order']):
            obj = sequence['objects'][obj_idx]
            deps_info = ""
            if obj['depends_on']:
                deps = [f"{dep['ref_type']} (via {dep['property']})" for dep in obj['depends_on']]
                deps_info = f" → depends on: {', '.join(deps)}"
            
            doc_content.append(f"{order_idx + 1}. **{obj['type']}**{deps_info}")
    
    # Reference patterns analysis
    doc_content.append("\n## Reference Patterns Analysis")
    doc_content.append("\nCommon embedded reference patterns found:")
    
    for ref_prop, usages in sorted(ref_patterns.items()):
        stix_types = sorted(set([usage['stix_type'] for usage in usages]))
        doc_content.append(f"\n### `{ref_prop}`")
        doc_content.append(f"**Used by**: {', '.join(stix_types)}")
        doc_content.append(f"**Total usages**: {len(usages)}")
        
        # Show implementation implications
        if ref_prop.endswith('_refs'):
            doc_content.append("**Parameter type**: List of object references")
        else:
            doc_content.append("**Parameter type**: Single object reference")
    
    # Dependencies summary
    doc_content.append("\n## Dependency Patterns")
    doc_content.append("\nObject dependency relationships found:")
    
    for pattern, count in sorted(dependency_patterns.items(), key=lambda x: x[1], reverse=True):
        doc_content.append(f"- **{pattern}**: {count} occurrences")
    
    # Implementation roadmap
    doc_content.append("\n## Implementation Roadmap")
    doc_content.append("\n### Phase 1: Implement Data Form Compatible Objects")
    doc_content.append("1. Create data form templates for all mappable STIX types")
    doc_content.append("2. Implement Python blocks with proper parameter extraction")
    doc_content.append("3. Test with simple objects first (no dependencies)")
    
    doc_content.append("\n### Phase 2: Handle Complex Dependencies")
    doc_content.append("1. Implement objects with embedded references")
    doc_content.append("2. Create orchestration notebooks for multi-object sequences")
    doc_content.append("3. Test dependency chains from examples")
    
    doc_content.append("\n### Phase 3: Expand Coverage")
    doc_content.append("1. Create directories for unmappable STIX types")
    doc_content.append("2. Implement remaining object types")
    doc_content.append("3. Validate against all example files")
    
    # Reconstitution specifications
    doc_content.append("\n## Reconstitution Specifications")
    doc_content.append("\nDetailed instructions for reconstructing original embedded reference structures:")
    
    # Multi-object reconstruction sequences
    multi_object_specs = {f: spec for f, spec in reconstitution_specs.items() if spec['type'] == 'multi_object'}
    if multi_object_specs:
        doc_content.append("\n### Multi-Object Reconstruction Sequences")
        
        for filename, spec in multi_object_specs.items():
            doc_content.append(f"\n#### {filename}")
            doc_content.append(f"**Objects**: {len(spec['objects'])}")
            doc_content.append(f"**Reconstruction Steps**: {len(spec['python_block_calls'])}")
            
            for call_spec in spec['python_block_calls']:
                doc_content.append(f"\n**Step {call_spec['step']}**: Create `{call_spec['object_type']}`")
                doc_content.append(f"- **Function**: `{call_spec['function_name']}()`")
                doc_content.append(f"- **Data Form**: `{call_spec['data_form_path']}`")
                doc_content.append(f"- **Variable**: `{call_spec['variable_name']}`")
                doc_content.append(f"- **Object ID**: `{call_spec['object_id']}`")
                
                if call_spec['reference_parameters']:
                    doc_content.append("- **Reference Parameters**:")
                    for ref_prop, ref_info in call_spec['reference_parameters'].items():
                        if ref_info['type'] == 'list':
                            doc_content.append(f"  - `{ref_prop}`: List of {len(ref_info['ref_ids'])} objects")
                        else:
                            doc_content.append(f"  - `{ref_prop}`: Single object reference")
                else:
                    doc_content.append("- **Reference Parameters**: None")
    
    # Single object specifications
    single_object_specs = {f: spec for f, spec in reconstitution_specs.items() if spec['type'] == 'single_object'}
    if single_object_specs:
        doc_content.append("\n### Single Object Reconstruction")
        
        for filename, spec in single_object_specs.items():
            doc_content.append(f"\n#### {filename}")
            if spec['python_block_call']:
                call_spec = spec['python_block_call']
                doc_content.append(f"**Function**: `{call_spec['function_name']}()`")
                doc_content.append(f"**Data Form**: `{call_spec['data_form_path']}`")
                doc_content.append(f"**Object Type**: `{call_spec['object_type']}`")
                
                if call_spec['reference_parameters']:
                    doc_content.append("**External Dependencies**:")
                    for ref_prop, ref_info in call_spec['reference_parameters'].items():
                        doc_content.append(f"- `{ref_prop}`: {ref_info['note']}")
                else:
                    doc_content.append("**External Dependencies**: None")
            else:
                doc_content.append("**Status**: No directory mapping available")
    
    # Python function signatures
    doc_content.append("\n### Python Block Function Signatures")
    doc_content.append("\nExpected function signatures for each STIX type:")
    
    function_signatures = {}
    for spec in reconstitution_specs.values():
        if spec['type'] == 'multi_object':
            for call_spec in spec['python_block_calls']:
                func_name = call_spec['function_name']
                ref_params = list(call_spec['reference_parameters'].keys())
                if func_name not in function_signatures:
                    function_signatures[func_name] = set(ref_params)
                else:
                    function_signatures[func_name].update(ref_params)
        elif spec['python_block_call']:
            call_spec = spec['python_block_call']
            func_name = call_spec['function_name']
            ref_params = list(call_spec['reference_parameters'].keys())
            if func_name not in function_signatures:
                function_signatures[func_name] = set(ref_params)
            else:
                function_signatures[func_name].update(ref_params)
    
    for func_name, ref_params in sorted(function_signatures.items()):
        params_str = ', '.join(['data_form'] + sorted(ref_params))
        doc_content.append(f"\n```python")
        doc_content.append(f"def {func_name}({params_str}):")
        doc_content.append(f"    \"\"\"")
        doc_content.append(f"    Create {func_name.replace('create_', '').replace('_', '-')} object")
        doc_content.append(f"    ")
        doc_content.append(f"    Args:")
        doc_content.append(f"        data_form: Clean template without embedded references")
        for param in sorted(ref_params):
            doc_content.append(f"        {param}: Referenced object(s)")
        doc_content.append(f"    ")
        doc_content.append(f"    Returns:")
        doc_content.append(f"        Complete STIX object with embedded references")
        doc_content.append(f"    \"\"\"")
        doc_content.append(f"    # Implementation needed")
        doc_content.append(f"    pass")
        doc_content.append(f"```")
    
    # Technical notes
    doc_content.append("\n## Technical Implementation Notes")
    doc_content.append("\n### Data Form Template Structure")
    doc_content.append("- Remove `id`, `spec_version`, `created`, `modified` (auto-generated)")
    doc_content.append("- Extract all `_ref` and `_refs` fields")
    doc_content.append("- Preserve all other object properties")
    doc_content.append("- Maintain STIX 2.1 compliance in core properties")
    
    doc_content.append("\n### Python Block Function Signatures")
    doc_content.append("- First parameter: data form template")
    doc_content.append("- Additional parameters: extracted reference objects")
    doc_content.append("- Parameter names match extracted reference field names")
    
    doc_content.append("\n### Orchestration Patterns")
    doc_content.append("- Create referenced objects first")
    doc_content.append("- Pass object references to dependent object creation")
    doc_content.append("- Use topological ordering for complex dependency chains")
    
    doc_content.append("\n### Reconstitution Process")
    doc_content.append("1. Load data form template from generated `_dataform.json` file")
    doc_content.append("2. Create or retrieve referenced objects based on extracted `_ref`/`_refs`")
    doc_content.append("3. Call Python block function with data form + reference parameters")
    doc_content.append("4. Function reconstructs original object with embedded references")
    doc_content.append("5. Store object in registry for subsequent object creation")
    
    return "\n".join(doc_content)

# Generate the documentation
orchestration_doc = generate_orchestration_documentation()

# Save to file
doc_path = orchestration_path / "STIX_Examples_to_DataForms_Guide.md"
with open(doc_path, 'w', encoding='utf-8') as f:
    f.write(orchestration_doc)

print(f"Orchestration documentation saved to: {doc_path}")
print(f"Document length: {len(orchestration_doc)} characters")

# Display first part of the document
print("\nDocument preview (first 2000 characters):")
print("=" * 50)
print(orchestration_doc[:2000] + "..." if len(orchestration_doc) > 2000 else orchestration_doc)

Orchestration documentation saved to: c:\projects\brett_blocks\Orchestration\STIX_Examples_to_DataForms_Guide.md
Document length: 23771 characters

Document preview (first 2000 characters):
# STIX Examples to Data Forms Conversion Guide

*Generated automatically from examples analysis*

**Analysis Date**: 2025-11-03 13:47:24
**Total Example Files**: 60
**STIX Object Types**: 37

## Overview

This document records the conversion of STIX examples from the `Block_Families/examples` directory into data form templates compatible with the Brett Blocks template-driven architecture.

### Key Principles:
- **Data Forms**: Clean templates with no embedded references
- **Extracted References**: Handled as separate function parameters
- **Object Sequences**: Dependency-ordered creation for complex scenarios

## Data Form Templates Generated

**Summary**: 90 mappable templates, 17 unmappable templates

### Successfully Mapped Templates

#### ARTIFACT
**Directory**: `Block_Families/StixORM/SCO/Artif

## Summary and Next Steps

This notebook has successfully analyzed all STIX examples and created a comprehensive conversion plan. The key outputs are:

1. **Data Form Templates**: Clean templates without embedded references
2. **Reference Extraction**: Systematic identification of `_ref` and `_refs` fields  
3. **Creation Sequences**: Dependency-ordered object creation patterns
4. **Directory Mapping**: Placement strategy for all template types
5. **Orchestration Guide**: Complete documentation for implementation

### Ready for Implementation:
- Run this notebook to generate the orchestration documentation
- Use the documentation to guide Python block development
- Implement data form templates in their respective directories
- Create orchestration notebooks for complex multi-object scenarios

The template-driven architecture can now be fully implemented with proper separation between data forms and embedded references.